In [23]:
import numpy as np
import pandas as pd
import globl
import re
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,accuracy_score,recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from KaggleWord2VecUtility import KaggleWord2VecUtility
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk import bigrams
from sklearn.linear_model import LogisticRegression, SGDClassifier,Lasso
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
import xgboost as xgb
import lightgbm as lgbm
from nltk.corpus import wordnet
from keras import optimizers
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LassoCV
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

path = "/home/pramod/Downloads/temp/romneycsv.csv"
data2 = pd.read_csv(path)

data2 = data2.drop(data2.query('Class!="1" & Class!="-1" & Class!="0"').index)

data2.to_csv('/home/pramod/Downloads/temp/romneyupdate.csv', index=False)

data1= pd.read_csv('/home/pramod/Downloads/temp/romneyupdate.csv')

d = {'date': data1["date"], 'tweet': data1["Anootated tweet"],'Class': data1["Class"]}
data = pd.DataFrame(data=d)

clean_train_reviews = []

for i in xrange(0,len(data)):
    clean_train_reviews.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(data["tweet"][i],False,True,False,False)))

    
    

clean_train_reviews1=[]
for review in clean_train_reviews:
    clean_train_reviews1.append(" ".join(KaggleWord2VecUtility.useless(review)))
    
text_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,4),min_df=1,strip_accents='unicode')),
                     #('feature_selection', SelectFromModel(svm.LinearSVC(penalty="l1",dual=False))),
                     #('feature_selection', SelectFromModel(RandomForestClassifier())),
                     ('clf', svm.LinearSVC(class_weight= 'balanced'))
                    ])


text_clf = text_clf.fit(clean_train_reviews1,data["Class"][0:len(data)])


5647


In [24]:
#vocab= text_clf.named_steps['vect'].vocabulary_
from sklearn import model_selection
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

y_pred=cross_val_predict(estimator=text_clf, X=clean_train_reviews1, y=data["Class"][0:len(data)],cv=StratifiedKFold(10))

print(classification_report(data["Class"][0:len(data)], y_pred))

             precision    recall  f1-score   support

         -1       0.70      0.76      0.73       631
          0       0.41      0.43      0.42       261
          1       0.61      0.44      0.51       235

avg / total       0.62      0.62      0.61      1127



In [4]:
data3 = pd.read_excel('/home/pramod/Downloads/sample-testdata.xlsx')
tweets=np.array(pd.DataFrame(data3["Anootated tweet"]))
tweets=tweets[1:]


'''data3= pd.read_csv('/home/pramod/Downloads/temp/obamaupdate.csv')
#d3 = {'date': data3["date"], 'tweet': data3["Anootated tweet"]}
#data_test = pd.DataFrame(data=d3)'''

clean_test = []

for i in xrange(0,len(tweets)):
    clean_test.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(tweets[i],False,True,False,False)))
test_labels=text_clf.predict(clean_test)
print test_labels[0:10]
test_labels=pd.DataFrame(test_labels)

#print test_labels
test_labels.to_csv('csvfile_romney.csv', encoding='utf-8', index=False,header=None)

[ 0  0 -1 -1 -1 -1  1  1 -1 -1]
